In [1]:
from utils import *
from model_util import *

In [2]:
model, tokenizer = load_model_and_tokenizer('meta-llama/Llama-2-7b-hf')

Loading model meta-llama/Llama-2-7b-hf


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Tokenizer padding side: left
Model and tokenizer for meta-llama/Llama-2-7b-hf loaded successfully


In [76]:
def generate_prompts(dataset_name, k, num_eval_items=5000):
    dataset = load_dataset(dataset_name)
    train_set = dataset['train']
    eval_set = dataset['validation']

    prompts, answers = [], []

    # load icl indices
    icl_indices = load_icl_indices(k)
    for eval_idx in range(num_eval_items):
        prompt = ""
        indices = icl_indices[str(eval_idx)]

        for idx in indices:
            prompt += train_set[idx]['combined'] + "\n\n"

        query = eval_set[eval_idx]['input'] + "\n"
        answer = eval_set[eval_idx]['output']

        prompt += query
        prompts.append(prompt)
        answers.append(answer)

    return prompts, answers


In [90]:
# prepare prompts
prompts, answers = generate_prompts('mnli', 10)

Loading dataset from datasets/mnli.json
Dataset loaded successfully from datasets/mnli.json
Loading ICL indices from data_indices/icl_indices_10_shot.json
ICL indices loaded successfully from data_indices/icl_indices_10_shot.json


In [91]:
torch.cuda.empty_cache()

In [103]:
print(prompts[7])
print("-----------")
print(answers[7])

And the boy is still very young"  Are we justified in saying that "THe girl is old."? Yes, no, or maybe?
No

John Knox was a tenant here from 1651 to 1672, but the house was owned by James Mosman, a royal goldsmith, and his wife Mariota Arres; a plaque bearing their initials can be seen on the outer wall. Are we justified in saying that "Houses were only owned by the people who lived in them in the 1600s."? Yes, no, or maybe?
No

Permanently boosting the rate of GDP growth would require ever-increasing relative shares of saving and Are we justified in saying that "Savings would have to increase more and more in order to boost the rate of GDP growth."? Yes, no, or maybe?
Yes

A variety of themes are space, health, communications, agriculture, etc. Are we justified in saying that "The themes change occasionally."? Yes, no, or maybe?
Maybe

The following year, acting with what Prime Minister Antonio Salandra acknowledged to be  sacro egoismo,  Italy signed a secret treaty to enter the war

In [92]:
# run activations using regular hf api
device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer.pad_token = tokenizer.eos_token
print(f"padding side: {tokenizer.padding_side}")


inputs = tokenizer([prompts[7]], padding=True, return_tensors="pt").to(device)
outputs = model(**inputs)
logits = outputs.logits

padding side: left


In [93]:
# run activations using custom method
print(logits.shape)

torch.Size([1, 684, 32000])


In [94]:
final_token_logits = logits[0, -1, :]

In [95]:
print(final_token_logits.shape)

torch.Size([32000])


In [96]:
# labels = ['Sports', 'Business', 'Science and technology', 'World politics']
labels = ['Yes', 'No', 'Maybe']

In [97]:
top_values, top_indices = torch.topk(final_token_logits, 4)

In [98]:
top_values

tensor([24.3727, 24.2131, 23.3707, 19.7781], device='cuda:0',
       grad_fn=<TopkBackward0>)

In [99]:
for index in top_indices:
    print(tokenizer.decode(index))

No
Maybe
Yes
Pro


In [100]:
print(top_indices)

tensor([ 3782, 22762,  8241,  1184], device='cuda:0')


In [101]:
for label in labels:
    encoded_text = tokenizer.encode(label, add_special_tokens=False)
    print(encoded_text)
    print(tokenizer.decode(encoded_text))
    
    logit_value = final_token_logits[encoded_text[0]]
    print(f"logit value: {logit_value}")
    
    # Apply softmax along the appropriate dimension (dim=0 here since logit_value is likely 1D)
    softmaxed_logit_value = torch.softmax(torch.Tensor([logit_value]), dim=0)
    print(f"softmaxed logit value: {softmaxed_logit_value}")
    
    print("---------")

[3869]
Yes
logit value: 11.518129348754883
softmaxed logit value: tensor([1.])
---------
[1939]
No
logit value: 8.69173812866211
softmaxed logit value: tensor([1.])
---------
[7198]
Maybe
logit value: 14.872579574584961
softmaxed logit value: tensor([1.])
---------


In [110]:
generate_new_texts(model, tokenizer, texts=[prompts[7]])

['Maybe\n\nthe the']

# investigating the outputs

In [75]:
import json
from pathlib import Path



model_name = "meta-llama/Llama-2-7b-hf"
dataset_name = "mnli"
k = 10


# load the generations
generations_path = Path(f"results/icl/accuracy_results/{model_name}/{dataset_name}/{k}-shot/generations.json")

with open(generations_path, 'r') as f:
    generations_data = json.load(f)


# load the logit data
logit_data_path = Path(f"results/icl/logits/{model_name}/{dataset_name}/{k}-shot/logit_data.json")

with open(logit_data_path, 'r') as f:
    logit_data = json.load(f)

for i in range(len(generations_data)):
    generation_correct = generations_data[i]['score']
    logit_accurate = logit_data[str(i)]['is_correct']
    
    if generation_correct and not logit_accurate:
        print(i)
        
        print()
        print(generations_data[i])
        print()
        print(logit_data[str(i)])
        
        break

7

{'prompt': 'And the boy is still very young"  Are we justified in saying that "THe girl is old."? Yes, no, or maybe?\nNo\n\nJohn Knox was a tenant here from 1651 to 1672, but the house was owned by James Mosman, a royal goldsmith, and his wife Mariota Arres; a plaque bearing their initials can be seen on the outer wall. Are we justified in saying that "Houses were only owned by the people who lived in them in the 1600s."? Yes, no, or maybe?\nNo\n\nPermanently boosting the rate of GDP growth would require ever-increasing relative shares of saving and Are we justified in saying that "Savings would have to increase more and more in order to boost the rate of GDP growth."? Yes, no, or maybe?\nYes\n\nA variety of themes are space, health, communications, agriculture, etc. Are we justified in saying that "The themes change occasionally."? Yes, no, or maybe?\nMaybe\n\nThe following year, acting with what Prime Minister Antonio Salandra acknowledged to be  sacro egoismo,  Italy signed a sec